# <center> PySpark Streaming </center>

In [ ]:
!module list

In [ ]:
!mkdir ~/software
!cd ~/software; wget https://archive.apache.org/dist/spark/spark-2.1.0/spark-2.1.0-bin-hadoop2.7.tgz

In [ ]:
!cd ~/software; tar -xzf spark-2.1.0-bin-hadoop2.7.tgz

In [ ]:
import os
import subprocess

def generate_compute_file(node_file=os.environ['PBS_NODEFILE'], 
                         slaves_file='/home/lngo/software/spark-2.1.0-bin-hadoop2.7/conf/slaves'):
    
    node_list = subprocess.check_output(['cat', node_file]).decode("utf-8").strip().split('\n')
    node_list = sorted(list(set(node_list)))

    print("Executors: ") 
    with open(slaves_file, 'w') as slave_file_fp:
        for node in node_list:
            slave_file_fp.write('{}\n'.format(node))
            print(node)
generate_compute_file()

In [ ]:
!more ~/software/spark-2.1.0-bin-hadoop2.7/conf/slaves

In [ ]:
!sh ~/software/spark-2.1.0-bin-hadoop2.7/sbin/stop-all.sh
!echo '------------------------------------------------------'
!sh ~/software/spark-2.1.0-bin-hadoop2.7/sbin/start-all.sh

In [ ]:
import sys
import os
import json

sys.path.insert(0, '/home/lngo/software/spark-2.1.0-bin-hadoop2.7/python')
sys.path.insert(0, '/home/lngo/software/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip')

os.environ["SPARK_HOME"] = "/home/lngo/software/spark-2.1.0-bin-hadoop2.7/"
os.environ['PYSPARK_PYTHON'] = '/software/anaconda/4.2.0/bin/python'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0 pyspark-shell'

import pyspark
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

conf = pyspark.SparkConf()
conf.setMaster("spark://node1774:7077")
conf.set("spark.driver.memory","8g")
conf.set("spark.executor.memory","50g")
conf.set("spark.num.executors","4")
conf.set("spark.executor.cores","12")

sc = pyspark.SparkContext(conf=conf, appName='PythonStreaming')
ssc = StreamingContext(sc, 2)

In [ ]:
sc

In [ ]:
ssc

In [ ]:
brokers = 'node1774.palmetto.clemson.edu:9092'
topic = 'test'

def parseTweet(x):
    textContent = 'NoContent'
    try:
        jsonData = json.loads(json.loads(x))
        textContent = jsonData['text']
    except TypeError:
        textContent = 'TypeError'
    return textContent

kvs = KafkaUtils.createDirectStream(ssc, [topic],{'metadata.broker.list': brokers})
lines = kvs.map(lambda x: x[1])
counts = lines.map(parseTweet).flatMap(lambda line: line.split(' ')) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a+b)

counts.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2017-11-27 11:48:06
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:06
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:08
-------------------------------------------
"{\"created_at\":\"Mon Nov 27 16:48:05 +0000 2017\",\"id\":935188490043654144,\"id_str\":\"935188490043654144\",\"text\":\"RT @SportsCenter: Latest AP Poll:\\n1. Clemson\\n2. Oklahoma\\n3. Wisconsin\\n4. Auburn\\n5. Alabama\\n6. Georgia\\n7. Miami\\n8. Ohio State\\n9. Penn State\\u2026\",\"source\":\"\\u003ca href=\\\"http:\\/\\/twitter.com\\/download\\/iphone\\\" rel=\\\"nofollow\\\"\\u003eTwitter for iPhone\\u003c\\/a\\u003e\",\"truncated\":false,\"in_reply_to_status_id\":null,\"in_reply_to_status_id_str\":null,\"in_reply_to_user_id\":null,\"in_reply_to_user_id_str\":null,\"in_reply_to_screen_name\":null,\"user\":{\"id\":1036147424,\"id_str\":\"1

-------------------------------------------
Time: 2017-11-27 11:48:08
-------------------------------------------
(u'RT', 1)
(u'Oklahoma\n3.', 1)
(u'@YogiLego', 1)
(u'Clemson', 1)
(u'beat', 1)
(u'@SportsCenter:', 1)
(u'State\n9.', 1)
(u'Poll:\n1.', 1)
(u'the', 1)
(u'Auburn,', 1)
...

-------------------------------------------
Time: 2017-11-27 11:48:10
-------------------------------------------
"{\"created_at\":\"Mon Nov 27 16:48:02 +0000 2017\",\"id\":935188476089327623,\"id_str\":\"935188476089327623\",\"text\":\"RT @SportsCenter: Latest AP Poll:\\n1. Clemson\\n2. Oklahoma\\n3. Wisconsin\\n4. Auburn\\n5. Alabama\\n6. Georgia\\n7. Miami\\n8. Ohio State\\n9. Penn State\\u2026\",\"source\":\"\\u003ca href=\\\"http:\\/\\/twitter.com\\/download\\/iphone\\\" rel=\\\"nofollow\\\"\\u003eTwitter for iPhone\\u003c\\/a\\u003e\",\"truncated\":false,\"in_reply_to_status_id\":null,\"in_reply_to_status_id_str\":null,\"in_reply_to_user_id\":null,\"in_reply_to_user_id_str\":null,\"in_reply_to_screen

-------------------------------------------
Time: 2017-11-27 11:48:20
-------------------------------------------
(u'\U0001f62d\U0001f62d\U0001f62d', 1)
(u'https://t.co/Q0gmWQDBVx', 1)

-------------------------------------------
Time: 2017-11-27 11:48:22
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:22
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:24
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:24
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:26
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:26
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 11:48:28
-------------------------------------------

---------

To stop the Spark job, call `sc.stop()`

In [ ]:
ssc.stop()
sc.stop()